In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')

In [ ]:
#öncelikle bu kısımda gerekli kütüphane ve modülleri import ve from ile ekliyoruz
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
#from keras.datasets import dermnet
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
#import random
#np.random.seed(0)
#verisetimiz de 23 tane sınıf vardır her bir verinin boyutu 48*48 olarak ayarlanmıştır.
num_classes = 23
img_rows,img_cols = 48,48
batch_size = 32


In [ ]:
#Derin öğrenme uygulamalarında sınıflandırma yaparken modeli kurarken veri seti eğitim ve test
#olmak üzere ikiye ayrılır.Eğitim verisinde modeli kurarız test verisinde ise modeli degerlendiririz
#burada veriseti yolunu gösteriyoruz ve verileri ayırıyoruz
#verilerimiz 23 class ve 15557 eğitim verisi ve 4002 test/validation verisi olmak üzere 2 kısma ayrıldı
train_data_dir = '../input/dermnet/train'#eğitim
validation_data_dir = '../input/dermnet/test'#test

#rescale=1./255
#her dijital görüntü,0-255 aralığında değere sahip piksel tarafından oluşturulur. 0 siyah ve 255 beyazdır.
#255 maksimum piksel değeri olduğundan Yeniden ölçeklendirme 1./255, [0,255] -> [0,1] aralığındaki her piksel değerini dönüştürmektir.

train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

#eğitim verileri ile ilgili bilgiler
#Found 15557 images belonging to 23 classes.
train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)
#doğrulama /test
#Found 4002 images belonging to 23 classes.

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            color_mode='grayscale',
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical',
                            shuffle=True)



In [ ]:
#plt.figure(0,figsize=(12,0))
#for i in range(1,13):
   # plt.subplot(3,4,i)
   # plt.axis("off")
    # image=train_datagen[i],reshape(48,48,3)
    #plt.imshow(image,cmap='gray')
   # print("train_datagen:",i)
    
    #plt.tight_layout()
    #plt.show()


In [ ]:
#Keras kütüphanesiyle sıralı katmanlardan oluşan sinir ağı modeli kullanırız.Model katman oluşturmamıza izin verir
model = Sequential()
#modele katman oluşturmak eklemmek için add() işlevini kullanırız.Öncelikle Conv2D katmanına uyguladık
#Conv2D 2 piksellik bir küçültme sağlar.
#MaxPooling kullanılarak kernel boyutlarında en büyük sayı alınmıştır
#pooling ile gösterimin kayma boyutunu ağ içindeki parametreleri ve hesaplama sayısını azaltmak için kullandıkBununlada ağdaki uyumsuzluk kontrol edildi.
#Maxpool2D İLE HER İKİ YÖNDEN BOYUTLAR YARIYA İNDİRİLDİ
#Sinir ağı eğitimi konusunda en iyi sonucu relu fonksiyonu veriyor Bu yüzden burda aktivasyon fonksiyonumuz relu
# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#2 katmanımızda da önce Conv2D uyguladık verimiz 48,48 boyutlu burada activasyon  fonksiyonu olarak elu kullandık
#Düzenli ve stabil eğitim için BatchNormalization() katmanını uyguladık.
#Bununla eğitim süresini azaltabilir ve modelin daha iyi performans göstermesi sağlanabilir.

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Diğer katmanlarda da yine benzer işlemler tekrarlanıyor.Burada dikkat etmemiz gereken bir diğer katman dropout()
#Dropout()ile 0.20 giriş değeri devre dışı bırakılıyor.
#ilk Conv katmanında görüntünün Width ve height değerleri mutlaka girilmelidir.Bu degerlere göre filtreleme işlemi yapılacaktır.
#

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Son olarak Flatten ve dense uygulanır.
#Flatten katmanı tek boyutlu diziye dönüştürür.
#Dense katman tür anlamına gelmektedir.
#Derin öğrenme yöntemlerinin doğrusal olmayan ilişkileri modellemesi için her düğümde gerçekleşen işlemler doğrusal olmayan bir akt.fonksiyonundan geçirilir.
#Akt. fonksiyonu ağırlıklı toplamı hesaplar ve nöronun aktive edilip edilmeyeceğine karar verir.
# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())




In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('skin_disease.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

#modeli derlerken compile fonksiyonunu kullanırız.Bir çok parametre alır fakat biz burada 3 tane kullandık.
#optimizer öğrenme oranı->Adam en yaygın kullanılır.
#metric için accuracy-doğruluk değerini verdik
#kayıp fonksiyonu loss için ise categorical_crossentropy kullandık.
#Yapay Sinir Ağları çalışmalarında maliyet fonksiyonu önemlidir ve minimum olması gerekir.

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

#Epochs devir sayısı:Bu değeri 25 olarak belirledik verisetinin kaç kez model üzerinden geçip eğitileceğini gösterir
#sayıyı küçük verirsek eğitim kısa sürer.
#batchsize aynı anda eğitilen veri sayısıdır.yani burda 4002 tane veriyi aynı anda eğitime sokmuş oluruz.

nb_train_samples = 15557
nb_validation_samples = 4002
epochs=25
#Validation doğrulama seti anlamına gelmektedir.Eğitimin model performansını test etmek için kullanılmaktadır.
#Model eğitilirken fit () işlevini kullanıyoruz
###model çalıştır###
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('Agg')
 
#aktivasyon fonksiyonları, activation functions
 
    
def step(x):
    return np.array(x > 0, dtype=np.int)
 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
 
def relu(x):
    return np.maximum(0, x)
 
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
 
def softplus(x):
    return np.log(1.0 + np.exp(x))
 
def tanh(x):
    return np.tanh(x)
 
def swish(x):
    #return x*sigmoid(x)
    return x*(1 / (1 + np.exp(-x)))
 
def prelu(x,alpha):
    a = []
    for item in x:
        if item < 0:
            a.append(alpha*item)
        else:
            a.append(item)
    return a
 
def elu(x,alpha):
    a = []
    for item in x:
        if item >= 0:
            a.append(item)
        else:
            a.append(alpha * (np.exp(item)-1))
    return a
 
x = np.arange(-5., 5., 0.1)
 
step=step(x)
sigmoid=sigmoid(x)
relu=relu(x)
softmax=softmax(x)
softplus=softplus(x)
tanh=tanh(x)
prelu=prelu(x,0.1)
elu=elu(x,1.0)
swish=swish(x)
 
#grafik çizimleri, plotting
 
plt.figure()
plt.xlabel("Girdiler")
plt.ylabel("Fonksiyon Çıktıları")
plt.grid(True)
plt.plot(x,step, label="Step", color='C0', lw=3)
plt.plot(x,sigmoid, label="Sigmoid", color='C1', lw=3)

plt.plot(x,softmax, label="Softmax", color='C3', lw=3)
plt.plot(x,softplus, label="SoftPlus", color='C4', lw=3)
plt.plot(x,tanh, label="TanH", color='C5', lw=3)
plt.plot(x,prelu, label="PReLU", color='C6', lw=3)
plt.plot(x,elu, label="ELU", color='C8', lw=3)
plt.plot(x,swish, label="Swish", color='C9', lw=3)
plt.legend()
plt.show()